In [1]:
import pandas as pd
import torch
import os
import sys
sys.path.insert(0,'../code/data_utils/')
from ConfigDataset import ConfigDataset

In [9]:
all_config_datasets = {}
for cell_number in range(1,18):
    if cell_number == 8:
        continue
    all_config_datasets[cell_number] = {}
    for replicate in range(3):
        all_config_datasets[cell_number][replicate] = {}
        for chrom in [*[str(k) for k in range(1,23)],'X']:
            try:
                all_config_datasets[cell_number][replicate][chrom] = ConfigDataset(
                    '../data/processed_data.hdf5',
                    segment_length=64,
                    remove_diagonal=False,
                    batch_size=0,
                    normalize_distances=False,
                    geos=None,
                    organisms=None,
                    cell_types=None,
                    cell_numbers=[cell_number],
                    chroms=[chrom],
                    replicates=[replicate],
                    shuffle=True,
                    allow_overlap=True,
                    two_channels=False,
                    try_GPU=False,
                    mean_dist_fp=None,
                    mean_sq_dist_fp=None
                )
            except:
                continue
            

'''
all_config_datasets = {
    cell_number:{
        replicate:{
            chrom:ConfigDataset(
                '../data/processed_data.hdf5',
                segment_length=64,
                remove_diagonal=False,
                batch_size=0,
                normalize_distances=False,
                geos=None,
                organisms=None,
                cell_types=None,
                cell_numbers=[cell_number],
                chroms=['1','2'],
                replicates=[replicate],
                shuffle=True,
                allow_overlap=True,
                two_channels=False,
                try_GPU=False,
                mean_dist_fp=None,
                mean_sq_dist_fp=None
            ) for chrom in [*[str(k) for k in range(1,23)],'X']
        } for replicate in range(3)
    } for cell_number in range(1,18) if cell_number != 8
}
'''

"\nall_config_datasets = {\n    cell_number:{\n        replicate:{\n            chrom:ConfigDataset(\n                '../data/processed_data.hdf5',\n                segment_length=64,\n                remove_diagonal=False,\n                batch_size=0,\n                normalize_distances=False,\n                geos=None,\n                organisms=None,\n                cell_types=None,\n                cell_numbers=[cell_number],\n                chroms=['1','2'],\n                replicates=[replicate],\n                shuffle=True,\n                allow_overlap=True,\n                two_channels=False,\n                try_GPU=False,\n                mean_dist_fp=None,\n                mean_sq_dist_fp=None\n            ) for chrom in [*[str(k) for k in range(1,23)],'X']\n        } for replicate in range(3)\n    } for cell_number in range(1,18) if cell_number != 8\n}\n"

In [10]:
len(all_config_datasets)

16

In [11]:
n = 0
for cell_number in all_config_datasets:
    for replicate in all_config_datasets[cell_number]:
        for chrom in all_config_datasets[cell_number][replicate]:
            n+= 1

In [15]:
raw_tan_data_dir = '../data/tan_single-cell_2018/'


def get_cell_number(fp):
    return int( fp.split('/')[-1].split('_')[-1].split('-')[0] )

# Find the directories that actually contain 
all_dirs = [ raw_tan_data_dir + d for d in os.listdir( raw_tan_data_dir ) ]
valid_dirs = []
for d in all_dirs:

    files = os.listdir(d)
    contains_clean = False
    for f in files: 
        if 'impute3.round4.clean.3dg' in f:
            contains_clean = True
            break

    if contains_clean:
        valid_dirs.append(
            (
                get_cell_number(d),
                d
            )
        )
valid_dirs.sort()


In [16]:
def get_raw_coords(replicate_number,directory):

    if directory[-1] != '/':
        directory+= '/'

    clean_structure_files = [f for f in os.listdir(directory) if 'impute3.round4.clean.3dg' in f]

    for f in clean_structure_files:

        
        if '_rep1_' in f:
            if replicate_number == 1:
                filepath = directory + f
                break
        elif '_rep2_' in f:
            if replicate_number == 2:
                filepath = directory + f
                break
        elif replicate_number == 0:
            filepath = directory + f

    coord_df = pd.read_csv(
        filepath,
        sep='\t',
        header=None,
        names=['Chromosome','Genomic_Index','x','y','z']
    )
    return coord_df

In [23]:
valid_dirs = [
    valid_dirs[0]
]

In [24]:
valid_dirs

[(1, '../data/tan_single-cell_2018/Supp_GSM3271347_GM12878_Cell_1')]